In [ ]:
import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go

cf.go_offline(connected= True)

In [ ]:
df = pd.read_csv('okt_test.csv',encoding='cp949').iloc[1:,1:]

In [ ]:
df

,Column2,Column3
1,"['[', '설빙', '](', '설빙', ')', '설빙', '디지털', '금액'...","['설빙', '설빙', '설빙', '디지털', '금액']"
2,"['[', '아웃백', '스테이크하우스', ']', '아웃백', '디지털', '5만...","['아웃백', '스테이크하우스', '아웃백', '디지털']"
3,"['(', '파리', '바게뜨', ')', '블레싱', '초코', '베리', '케이크']","['파리', '바게뜨', '블레싱', '초코', '베리', '케이크']"
4,"['[', '스타벅스', '](', '스타벅스', ')(', 'APP', '전용',...","['스타벅스', '스타벅스', '전용', '카드', '교환']"
5,"['(', '파리', '바게뜨', ')', '생', '딸기', '요거트', '케이크']","['파리', '바게뜨', '생', '딸기', '요거트', '케이크']"
...,...,...
10294,"['시원한', '스노우', '리플', '여름', '누비', '이불', '패드', '...","['스노우', '리플', '여름', '누비', '이불', '패드', '풀', '세트..."
10295,"['순면', '누빔', '고정', '밴딩', '여름', '침대', '패드', '/'...","['순면', '누빔', '고정', '밴딩', '여름', '침대', '패드', '인견..."
10296,"['[', '베드', '리움', '][', '금성', '침대', 'X', '베드',...","['베드', '리움', '금성', '침대', '베드', '리움', '베드', '리움..."
10297,"['네스트', '호텔', '인천', '(', '인천', '호텔', '/', '중구'...","['네스트', '호텔', '인천', '인천', '호텔', '중구', '인천', '국..."


# 한국어 단어 토큰화

In [ ]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()

print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요")) 

OKT 형태소 분석 : ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
OKT 품사 태깅 : [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
OKT 명사 추출 : ['코딩', '당신', '연휴', '여행']


In [ ]:
df1 = pd.read_csv('product.csv',header = None)

In [ ]:
df_title =  df1.iloc[:,[3]]
df_title.columns = ['title']

중복 상품명 제거

In [ ]:
df_title = df_title.drop_duplicates()

In [ ]:
df_title

,title
0,(파리바게뜨) 실키롤케익
1,[설빙](설빙) 설빙 디지털 금액권 2만원권
2,[아웃백스테이크하우스]아웃백 디지털 5만원권
3,(파리바게뜨) 블레싱 초코베리 케이크
4,[스타벅스](스타벅스)(APP전용) e카드 3만원 교환권
...,...
10294,시원한스노우리플여름누비이불패드풀세트SSQK사이즈
10295,순면누빔 고정밴딩 여름침대패드/인견패드
10296,[베드리움][금성침대 X 베드리움] [베드리움] 슈페르 프리미엄 300T 매트리스(...
10297,네스트호텔 인천(인천 호텔/중구/인천국제공항(중구))


명사 추출

In [ ]:
from tqdm import tqdm

word_count = {}


for text in tqdm(df_title['title']):
    for word in okt.nouns(text):
        word_count[word] = word_count.get(word,0) + 1
        
    

100%|████████████████████████████████████████████████████████████████████████████| 9968/9968 [00:07<00:00, 1386.51it/s]


In [ ]:
import operator

sdict = sorted(word_count.items(), key=operator.itemgetter(1),reverse=True)


길이가 1 이하인 단어 제거

In [ ]:
lst_word = []
for i,j in sdict:
    if len(i) <= 1:
        pass
    else:
        lst_word.append([i,j])

In [ ]:
import konlpy
from sklearn.feature_extraction.text import CountVectorizer

명사 추출 결과

In [ ]:
df_word =  pd.DataFrame(lst_word,columns = ['word', 'num'])
df_word

,word,num
0,세트,624
1,삼성,369
2,전자,367
3,여름,313
4,남성,310
...,...,...
9128,바란,1
9129,예은,1
9130,파르페,1
9131,슈페르,1


In [ ]:
cv = CountVectorizer(vocabulary = df_word['word'],max_features= 3000,min_df= 3, max_df= 0.5)
print(cv)

CountVectorizer(max_df=0.5, max_features=3000, min_df=3,
                vocabulary=0        세트
1        삼성
2        전자
3        여름
4        남성
       ... 
9128     바란
9129     예은
9130    파르페
9131    슈페르
9132    거제시
Name: word, Length: 9133, dtype: object)


In [ ]:
df_title

,title
0,(파리바게뜨) 실키롤케익
1,[설빙](설빙) 설빙 디지털 금액권 2만원권
2,[아웃백스테이크하우스]아웃백 디지털 5만원권
3,(파리바게뜨) 블레싱 초코베리 케이크
4,[스타벅스](스타벅스)(APP전용) e카드 3만원 교환권
...,...
10294,시원한스노우리플여름누비이불패드풀세트SSQK사이즈
10295,순면누빔 고정밴딩 여름침대패드/인견패드
10296,[베드리움][금성침대 X 베드리움] [베드리움] 슈페르 프리미엄 300T 매트리스(...
10297,네스트호텔 인천(인천 호텔/중구/인천국제공항(중구))


In [ ]:
review_cv = cv.fit_transform(df_title['title'])

In [ ]:
review_cv

<9968x9133 sparse matrix of type '<class 'numpy.int64'>'
	with 22212 stored elements in Compressed Sparse Row format>

### DTM table 계산 계산 및 적용

첫 20개 특성으로 DTM 만들기

In [ ]:
print(cv.get_feature_names()[0:20])
print(review_cv.toarray())

['세트', '삼성', '전자', '여름', '남성', '블랙', '키즈', '여성', '박스', '신세계', '아이', '증정', '정품', '가방', '마스크', '무선', '화이트', '티셔츠', '프로', '아동']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


C:\Users\yoon3\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [ ]:
df_review_cv = pd.DataFrame(review_cv.toarray(),columns  =cv.get_feature_names() )

In [ ]:
cv = CountVectorizer(max_features= 9000,min_df= 2, max_df= 0.7,)

In [ ]:
test = cv.fit_transform(df_title['title'].tolist())

In [ ]:
df_test = pd.DataFrame(test.toarray(), columns= cv.get_feature_names())

C:\Users\yoon3\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [ ]:
df_test

,00,000,0001,000원,001,00198,002,003,004,005,...,히어로,히키스,힌지보호,힐링,힐링그린,힐링팩토리,힐튼,힙분유,힙색,힙팩
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9965,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 데이터 정규화 & 정제

숫자와 영어가 붙어있는 형태소를 제거하여 한글과 한자 형태소만 남기기

In [ ]:
import re

def hasNumber(stringVal):
    if (re.search('\d', stringVal) == None):
        return False
    else:
        return True
    
    
def hasEnglish(stringVal):
    if (re.search('[A-Za-z]',stringVal) == None):
        return False
    else:
        return True

In [ ]:
hasEnglish('a')

True

In [ ]:
col_lst = []

for col in df_test.columns:
    if (hasNumber(col)) or (hasEnglish(col)):
        pass
    else:
        col_lst.append(col)
    

In [ ]:
df_test_2 = df_test.loc[:,col_lst].copy()

In [ ]:
word_value = df_test_2.iloc[:,1:].apply(sum,axis=0)

In [ ]:
pd.DataFrame(word_value).sort_values(0,ascending=False)[0:1000].iplot('bar', title = '단어 별 발생 빈도 1000순위 까지')

In [ ]:
lst_len = []
for i in df_test_2.columns:
    lst_len.append(len(i))

In [ ]:
pd.Series(lst_len).value_counts().iplot('bar',title = '단어길이(x)에 따라 토큰화된 문자의 수')

In [ ]:
df_test_2.insert(0,'Main_category',df1[0])

In [ ]:
df_test_2

,Main_category,_편백나무,正品,防彈少年團,가게놀이,가격인하,가고,가그린,가나양말,가난한,...,히어로,히키스,힌지보호,힐링,힐링그린,힐링팩토리,힐튼,힙분유,힙색,힙팩
0,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9964,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9965,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9966,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import numpy as np
random = np.random.random_integers(0,9967,size = int(9967* 0.2)).tolist()
df_test_3 = df_test_2.drop(random,axis=0).copy()
df_test_3

C:\Users\yoon3\AppData\Local\Temp\ipykernel_4216\2579555577.py:2: DeprecationWarning:

This function is deprecated. Please call randint(0, 9967 + 1) instead



,Main_category,_편백나무,正品,防彈少年團,가게놀이,가격인하,가고,가그린,가나양말,가난한,...,히어로,히키스,힌지보호,힐링,힐링그린,힐링팩토리,힐튼,힙분유,힙색,힙팩
0,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9963,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9964,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9966,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


모든 값이 0인 row도 삭제한 후 DTM

In [ ]:
df_test_3.set_index('Main_category').apply(sum,axis=0)

_편백나무     2
正品        3
防彈少年團     2
가게놀이      2
가격인하      2
         ..
힐링팩토리     2
힐튼        3
힙분유       4
힙색       14
힙팩        3
Length: 5811, dtype: int64

In [ ]:
name_lst  = [i for i in [df_test_3.set_index('Main_category').apply(sum,axis=0) > 0][0]]

In [ ]:
df_test_4 = df_test_3.set_index('Main_category').copy()

In [ ]:
df_test_5 = df_test_4.loc[:,name_lst].copy()

나이브 베이즈 분류기 사용

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB() 

NB_clf.fit(x_train, y_train) 

print('Train set score: {:.3f}'.format(NB_clf.score(x_train, y_train))) 
print('Test set score: {:.3f}'.format(NB_clf.score(x_test, y_test))) 

Train set score: 0.720
Test set score: 0.493


In [ ]:
df_title_2

,title
Main_category,
e쿠폰/티켓,(파리바게뜨) 실키롤케익
e쿠폰/티켓,[설빙](설빙) 설빙 디지털 금액권 2만원권
e쿠폰/티켓,[아웃백스테이크하우스]아웃백 디지털 5만원권
e쿠폰/티켓,(파리바게뜨) 블레싱 초코베리 케이크
e쿠폰/티켓,[스타벅스](스타벅스)(APP전용) e카드 3만원 교환권
...,...
가구/침구,시원한스노우리플여름누비이불패드풀세트SSQK사이즈
가구/침구,순면누빔 고정밴딩 여름침대패드/인견패드
가구/침구,[베드리움][금성침대 X 베드리움] [베드리움] 슈페르 프리미엄 300T 매트리스(...


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_title_2,
                                                      df_title_2.index, test_size=0.2, shuffle=True, stratify=df_title_2.index
                                                     ,random_state=34)

In [ ]:
x_train['title'].values.tolist()

['소울키친 빅마마 이혜정의 시크릿코인 205개',
 '예술가의 서재  : 그들은 어떻게 책과 함께 살아가는가  니나 프루덴버거',
 'Principles of Mathematical Analysis  3/E  W. Rudin',
 'MONSTA X Mini Album SHAPE of LOVE - JEWEL SET',
 '[포레스트스토리]B7 약산성 쿨 탈모 샴푸 1000+1000ml+250ml+헤어에센스',
 '[닌텐도]스위치 조이콘 블루옐로우 V',
 '[보테가베네타](부산점)[노벨라] [보테가베네타] 22SS 여성 인트레치아토 패딩 테크 카세트 크로스백 (628951_VBO81_12...',
 '[오후출발5일/6일] 괌 온워드리조트+워터파크+조식포함',
 '[프랭키뉴욕]남자 린넨 팬츠 여름 마 바지 밴딩 빅사이즈 와이드 남성_ID211_1',
 '벌집형 매트 자동차 카매트 국내 전차종',
 '[던킨](던킨) 던킨 베스트셀러',
 '태양열 LED 가로등 조명 벽등 태양광 32구 정원등',
 'Bricks Listening 300-3  Curtis MacDonald Misty Kim Brandon Cusak Andrea M. Cox Andrew MacI...',
 '[ROG]ASUS ROG G713RM-KH053 빅스마일데이 최종가 138만',
 '[아이깨끗해]손세정제 4.5L 대용량',
 '[삼성전자]갤럭시 S22 울트라 256G 자급제 SM-S908N',
 '[메디케이알]새부리형 KF94 국내생산 100매 대형 화이트 (10x10)',
 '[랩 시리즈][랩시리즈][5][남자스킨] NEW 데일리 레스큐 워터로션 Size up 세트 (+60ml 추가 증정)',
 '[오멘]OMEN 16-b1030TX 게이밍노트북 최종155만/i7/RTX3060',
 '[아라홈]코어힙 자세 교정 기능성 방석 사무실 골반',
 '[홈플러스시그니처]홈플러스시그니처위생백소형100매17x25CM',
 '[이블랑]시어서커 쿨 토퍼/고밀도 워싱 토퍼/극세사 웜 토퍼',
 '[래핑차일드]래

## TF-IDF를 적용시킬 때 각각 따로 해야한다는 것을 잊지 말자.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(max_features=2000,) 

tfidf_title_2 = tfidf.fit_transform(df_title_2['title'].tolist()).toarray()

pd.DataFrame(tfidf_title_2, columns = tfidf.get_feature_names_out())

,000원,001,01,010,10,100,1000,10000엔,1000ml,100g,...,후지필름,휠라,휠라키즈,휴대용,휴지,흰색,히든밴딩,히키스,힐링,힙색
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_title_2.head()

,title
Main_category,
e쿠폰/티켓,(파리바게뜨) 실키롤케익
e쿠폰/티켓,[설빙](설빙) 설빙 디지털 금액권 2만원권
e쿠폰/티켓,[아웃백스테이크하우스]아웃백 디지털 5만원권
e쿠폰/티켓,(파리바게뜨) 블레싱 초코베리 케이크
e쿠폰/티켓,[스타벅스](스타벅스)(APP전용) e카드 3만원 교환권


In [ ]:
x_train

,title
Main_category,
식품,소울키친 빅마마 이혜정의 시크릿코인 205개
도서/음반,예술가의 서재 : 그들은 어떻게 책과 함께 살아가는가 니나 프루덴버거
도서/음반,Principles of Mathematical Analysis 3/E W. R...
도서/음반,MONSTA X Mini Album SHAPE of LOVE - JEWEL SET
화장품/헤어,[포레스트스토리]B7 약산성 쿨 탈모 샴푸 1000+1000ml+250ml+헤어에센스
...,...
생활/주방/건강,호텔수건 코마사 40수 200g 10장 세트 고급 수건
여행,여행 필수품 3단 방수 워시백 세면도구 파우치
패션의류,[팍티오]팍티오 썸머그랜드오픈/예쁜데일리룩/봄상품할인/무배


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2000,) 
X_train_tfidf = tfidf.fit_transform(x_train['title'].values.tolist()) # train set을 변환
a = tfidf.get_feature_names_out()
X_test_tfidf = tfidf.transform(x_test['title'].values.tolist()) # test set을 변환
b = tfidf.get_feature_names_out()

In [ ]:
y_train

Index(['식품', '도서/음반', '도서/음반', '도서/음반', '화장품/헤어', '컴퓨터/전자', '신발/잡화', '여행',
       '패션의류', '스포츠/자동차',
       ...
       '가구/침구', '식품', '식품', '화장품/헤어', 'e쿠폰/티켓', '생활/주방/건강', '여행', '패션의류',
       '생활/주방/건강', '가구/침구'],
      dtype='object', name='Main_category', length=7974)

In [ ]:
import re

def hasNumber(stringVal):
    if (re.search('\d', stringVal) == None):
        return True
    else:
        return False
    
    
def hasEnglish(stringVal):
    if (re.search('[A-Za-z]',stringVal) == None):
        return True
    else:
        return False

In [ ]:
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(),columns = tfidf.get_feature_names_out() )

In [ ]:
lst_bool = []
for i in X_train_tfidf_df.columns:
    if hasNumber(i) and hasEnglish(i):
        lst_bool.append(i)
        
    else:
        pass


In [ ]:
X_train_tfidf_df_2 = X_train_tfidf_df.loc[:,lst_bool].copy()

In [ ]:
X_train_tfidf_df_2.index = x_train.index

In [ ]:
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(),columns = tfidf.get_feature_names_out() )

In [ ]:
lst_bool = []
for i in X_test_tfidf_df.columns:
    if hasNumber(i) and hasEnglish(i):
        lst_bool.append(i)
        
    else:
        pass

In [ ]:
X_test_tfidf_df_2 = X_test_tfidf_df.loc[:,lst_bool].copy()

In [ ]:
X_test_tfidf_df_2.index = x_test.index

In [ ]:
X_test_tfidf_df_2

,가능,가디건,가리개,가리개커튼,가방,가벼운,가보시,가보흙침대,가성비,가정용,...,휠라,휠라키즈,휴대,휴대용,휴지,흰색,히든밴딩,히키스,힐링,힙색
Main_category,,,,,,,,,,,,,,,,,,,,,
컴퓨터/전자,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
유아동/출산,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
생활/주방/건강,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
화장품/헤어,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
컴퓨터/전자,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
유아동/출산,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
식품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 여러 모델 적용 


In [ ]:
import pandas as pd
import numpy as np # 각 모델에서 내부적으로 관련 라이브러리 사용 가능

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

from sklearn.neighbors import KNeighborsClassifier             # 1. K-Nearest Neighbor(KNN)
from sklearn.linear_model import LogisticRegression            # 2. Logistic Regression
from sklearn.svm import SVC                                                # 3. SVC
from sklearn.tree import DecisionTreeClassifier                   # 4. Decision Tree
from sklearn.ensemble import RandomForestClassifier       # 5. Random Forest
from sklearn.ensemble import ExtraTreesClassifier             # 6. Extra Tree
from sklearn.ensemble import GradientBoostingClassifier  # 7. GBM
from sklearn.naive_bayes import GaussianNB                     # 8. GaussianNB
from xgboost import XGBClassifier                                     # 9. XGBoost
from lightgbm import LGBMClassifier                                 # 10. LightGBM
import warnings

warnings.filterwarnings('ignore')

## TF-IDF 이용

### TF-IDF train, test set 분리

In [ ]:
X_train_tfidf = pd.read_csv('X_train_tfidf.csv')
X_test_tfidf = pd.read_csv('X_test_tfidf.csv')
y_train = pd.read_csv('y_train.csv',index_col= 0)
y_test = pd.read_csv('y_test.csv',index_col= 0)
TF = pd.read_csv('TF.csv')

In [ ]:
X_train_tfidf

,Main_category,가능,가디건,가리개,가리개커튼,가방,가벼운,가보시,가보흙침대,가성비,...,휠라,휠라키즈,휴대,휴대용,휴지,흰색,히든밴딩,히키스,힐링,힙색
0,식품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,화장품/헤어,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969,생활/주방/건강,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7970,여행,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7971,패션의류,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7972,생활/주방/건강,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_tfidf_2 = X_train_tfidf.set_index('Main_category').copy()

In [ ]:
X_train_tfidf_2

,가능,가디건,가리개,가리개커튼,가방,가벼운,가보시,가보흙침대,가성비,가정용,...,휠라,휠라키즈,휴대,휴대용,휴지,흰색,히든밴딩,히키스,힐링,힙색
Main_category,,,,,,,,,,,,,,,,,,,,,
식품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
화장품/헤어,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
생활/주방/건강,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
여행,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
패션의류,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_tfidf_2.loc[:,X_train_tfidf_2.iloc[:,:].apply(sum,axis=0) > 1]

,가능,가디건,가리개,가리개커튼,가방,가벼운,가보시,가보흙침대,가성비,가정용,...,휠라,휠라키즈,휴대,휴대용,휴지,흰색,히든밴딩,히키스,힐링,힙색
Main_category,,,,,,,,,,,,,,,,,,,,,
식품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
화장품/헤어,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
생활/주방/건강,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
여행,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
패션의류,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_tfidf_2

,가능,가디건,가리개,가리개커튼,가방,가벼운,가보시,가보흙침대,가성비,가정용,...,휠라,휠라키즈,휴대,휴대용,휴지,흰색,히든밴딩,히키스,힐링,힙색
Main_category,,,,,,,,,,,,,,,,,,,,,
식품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
도서/음반,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
화장품/헤어,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
생활/주방/건강,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
여행,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
패션의류,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_tfidf_2.reset_index().iloc[:,1:]
X_test_tfidf.iloc[:,1:]

,가능,가디건,가리개,가리개커튼,가방,가벼운,가보시,가보흙침대,가성비,가정용,...,휠라,휠라키즈,휴대,휴대용,휴지,흰색,히든밴딩,히키스,힐링,힙색
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y_train

,Main_category
0,식품
1,도서/음반
2,도서/음반
3,도서/음반
4,화장품/헤어
...,...
7969,생활/주방/건강
7970,여행
7971,패션의류
7972,생활/주방/건강


### Naive Bayes 분류기 적용

In [ ]:
NB_clf.fit(X_train_tfidf_df_2, y_train) 
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf_df_2.values, y_train))) 
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf_df_2.values, y_test))) 


Train set score: 0.734
Test set score: 0.686


### DecisionTree 분류기 적용

In [ ]:
from sklearn.tree import DecisionTreeClassifier


tree3 = DecisionTreeClassifier(min_samples_split=50,random_state=7)

tree3.fit(X_train_tfidf_df_2.values, y_train)
print('#Decision Tree train set score: {:.3f}'.format(tree3.score(X_train_tfidf_df_2.values, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(tree3.score(X_test_tfidf_df_2.values, y_test)))

#Decision Tree train set score: 0.768
#Decision Tree test set score: 0.632


### LogisticRegression 분류기 적용

In [ ]:
from sklearn.linear_model import LogisticRegression 
LR_clf = LogisticRegression() 

LR_clf.fit(X_train_tfidf_df_2.values, y_train) 
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf_df_2.values, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf_df_2.values, y_test))) 

#Train set score: 0.775
#Test set score: 0.708


### LSA

LSA를 통해 변수가 1000개인 Truncated SVD 형식으로 데이터를 변환

SVC 모델을 활용해서 정확도를 확인

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, random_state=7) #압축할 component의 수 지정
X_train_lsa = svd.fit_transform(X_train_tfidf_2.reset_index().iloc[:,1:])
X_test_lsa = svd.transform(X_test_tfidf.iloc[:,1:])

print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))

SVC_clf = SVC()
SVC_clf.fit(X_train_lsa, y_train)
print('#Train set score: {:.3f}'.format(SVC_clf.score(X_train_lsa, y_train)))
print('#Test set score: {:.3f}'.format(SVC_clf.score(X_test_lsa, y_test)))

LSA Converted X shape: (7974, 1000)
Sum of explained variance ratio: 0.908
#Train set score: 0.812
#Test set score: 0.700


truncated SVD 형식으로 변환된 train data를 확인

In [ ]:
pd.DataFrame(X_train_lsa)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000081,-0.000004,-0.000023,1.003534e-05,0.000015,-0.000001,-3.231149e-07,0.000011,0.000001,-0.000006,...,0.001146,-0.001129,0.003756,0.000361,0.002675,-0.000313,0.003694,0.001166,0.001610,-0.001370
2,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000335,0.001348,0.005392,-5.150541e-05,0.009983,-0.005031,-2.541684e-04,-0.001357,0.000110,-0.007799,...,-0.002330,0.005009,-0.000299,0.004141,-0.011010,0.003014,-0.003447,-0.001311,-0.012516,0.014796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969,0.254834,-0.006684,-0.023547,1.947777e-04,0.007932,-0.000875,-3.720488e-04,0.003933,0.000471,-0.003805,...,-0.004768,0.017529,0.004293,0.000619,0.018945,0.000143,-0.053118,0.008156,0.003624,0.001926
7970,0.000810,0.005094,0.013689,-1.608831e-04,0.016407,-0.003752,-3.779725e-04,-0.000432,-0.001118,0.027132,...,-0.000374,-0.002130,0.016399,-0.008929,-0.004956,0.007054,-0.014503,0.001188,-0.008638,-0.011542
7971,0.000435,0.006591,0.006457,1.604564e-04,0.003991,-0.000570,1.347470e-03,0.002115,-0.000054,0.007117,...,-0.001874,-0.011565,-0.001535,-0.001833,0.006106,-0.013156,-0.001339,0.004568,-0.000213,0.001640
7972,0.001468,0.003649,0.013065,-1.405143e-04,0.016442,-0.004820,2.746237e-04,0.000253,-0.000815,0.044325,...,-0.006209,0.005733,0.004953,0.015232,-0.010225,-0.003319,-0.001715,0.005660,-0.004562,0.007683


In [ ]:
pd.DataFrame(X_test_lsa)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.002016,0.016775,0.024137,-0.000213,0.049198,0.026640,-0.005330,-0.008859,0.001922,0.016138,...,-0.001286,0.000661,-0.005762,0.015950,-0.014207,-0.006008,0.007764,0.011544,-0.006788,-0.007751
1,0.019985,0.004230,0.178722,-0.001604,-0.132919,0.009704,-0.010316,-0.180617,-0.006148,0.099612,...,-0.000464,-0.009890,-0.007727,-0.001865,-0.004455,-0.003321,-0.006738,0.014094,0.008456,0.001826
2,0.018598,-0.000139,0.004898,-0.000048,0.000826,-0.000502,-0.000189,0.000116,0.000226,0.005431,...,-0.000880,-0.003047,0.001690,0.000803,-0.002470,0.003096,0.000574,-0.004445,-0.001289,-0.002990
3,0.006731,-0.000243,-0.000920,0.000010,0.000530,-0.000080,0.000025,0.000643,0.000022,-0.000284,...,-0.001446,-0.009937,0.003934,-0.005713,-0.004736,0.001564,0.000263,0.006173,-0.000633,0.008523
4,0.003336,0.001198,0.003852,-0.000021,0.008280,-0.004128,-0.000219,-0.000953,0.000027,-0.006691,...,-0.012512,0.028827,-0.027402,0.010990,0.005363,-0.023877,-0.015465,-0.009487,-0.003699,-0.002104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.006971,0.338458,-0.035748,0.000284,-0.035927,-0.007772,-0.007746,-0.011919,-0.000430,-0.003858,...,0.006752,-0.001221,-0.002404,0.011000,-0.012941,-0.010713,-0.006589,-0.000117,-0.007487,-0.002958
1990,0.000048,0.000107,0.001078,-0.000007,0.000109,-0.000029,-0.000055,-0.000466,0.000226,-0.000172,...,-0.017401,-0.012170,-0.001038,-0.002777,0.016799,-0.004228,-0.011939,-0.003069,-0.008761,-0.000845
1991,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1992,0.000147,0.000451,0.002920,-0.000028,0.002852,-0.001569,-0.000203,-0.001192,0.001436,-0.002372,...,0.008394,-0.006275,-0.006546,0.015010,-0.019427,-0.005539,0.010464,0.006161,-0.012994,-0.001785


In [ ]:
y_train['Main_category'].unique()

array(['식품', '도서/음반', '화장품/헤어', '컴퓨터/전자', '신발/잡화', '여행', '패션의류',
       '스포츠/자동차', 'e쿠폰/티켓', '가구/침구', '생활/주방/건강', '유아동/출산'], dtype=object)

In [ ]:
print('#사용된 전체 카테고리:', y_train['Main_category'].unique())
print('#첫 문서의 카테고리:', y_train['Main_category'][0])

#사용된 전체 카테고리: ['식품' '도서/음반' '화장품/헤어' '컴퓨터/전자' '신발/잡화' '여행' '패션의류' '스포츠/자동차' 'e쿠폰/티켓'
 '가구/침구' '생활/주방/건강' '유아동/출산']
#첫 문서의 카테고리: 식품


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


sim_result = cosine_similarity([X_train_lsa[0]], X_train_lsa) 
sim_result

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
print("#Top 20 유사도(lsa):\n", sorted(sim_result[0].round(2), reverse=True)[:20])
sim_index = (-sim_result[0]).argsort()[:20]
print('#Top 20 유사 뉴스의 인덱스(lsa):\n', sim_index)
sim_labels = [y_train['Main_category'][i] for i in sim_index]
print('#Top 20 유사 뉴스의 카테고리(lsa):\n', sim_labels)

#Top 20 유사도(lsa):
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
#Top 20 유사 뉴스의 인덱스(lsa):
 [   0 5323 5322 5321 5320 5319 5318 5317 5316 5315 5314 5313 5312 5311
 5310 5309 5308 5307 5306 5305]
#Top 20 유사 뉴스의 카테고리(lsa):
 ['식품', '가구/침구', '스포츠/자동차', '식품', '식품', '컴퓨터/전자', '컴퓨터/전자', '스포츠/자동차', '화장품/헤어', '유아동/출산', '도서/음반', '생활/주방/건강', '식품', '생활/주방/건강', '여행', '패션의류', '신발/잡화', '스포츠/자동차', '식품', '컴퓨터/전자']


### LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components = len(set(y_train['Main_category'])), doc_topic_prior= 0.1)

In [ ]:
TF_2 = TF.set_index('Main_category')

In [ ]:
TF_2.shape

(8166, 5716)

In [ ]:
LDA.fit(TF.iloc[:,1:])

LatentDirichletAllocation(doc_topic_prior=0.1, n_components=12)

In [ ]:
TF_3 = TF_2.loc[:,TF_2.apply(sum,axis= 0) > 5].copy()

In [ ]:
TF_2.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
TF_2

,_편백나무,正品,防彈少年團,가게놀이,가격인하,가고,가그린,가나양말,가난한,가능,...,히어로,히키스,힌지보호,힐링,힐링그린,힐링팩토리,힐튼,힙분유,힙색,힙팩
Main_category,,,,,,,,,,,,,,,,,,,,,
e쿠폰/티켓,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e쿠폰/티켓,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e쿠폰/티켓,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e쿠폰/티켓,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e쿠폰/티켓,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
가구/침구,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가구/침구,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가구/침구,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
TF_2.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
TF_lst = TF['Main_category'].value_counts().reset_index().reset_index()[['level_0','Main_category']].values

In [ ]:
TF_lst

array([[   0, 1246],
       [   1, 1075],
       [   2, 1048],
       [   3,  896],
       [   4,  749],
       [   5,  740],
       [   6,  638],
       [   7,  632],
       [   8,  496],
       [   9,  318],
       [  10,  207],
       [  11,  121]], dtype=int64)

In [ ]:
TF_lst

array([[   0, 1246],
       [   1, 1075],
       [   2, 1048],
       [   3,  896],
       [   4,  749],
       [   5,  740],
       [   6,  638],
       [   7,  632],
       [   8,  496],
       [   9,  318],
       [  10,  207],
       [  11,  121]], dtype=int64)

LDA에는 앞 과정에서 전처리한 TF Table을 사용했다.

각 class별로 관측치가 몇 번 나왔는지 count하여 a라는 변수에 저장했다.

In [ ]:
a = pd.DataFrame(TF_lst)[1].tolist()

In [ ]:
a

[1246, 1075, 1048, 896, 749, 740, 638, 632, 496, 318, 207, 121]

In [ ]:
TF_2.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
from gensim.models.ldamodel import LdaModel


common_corpus의 구조를 확인

In [ ]:
from gensim.test.utils import common_corpus

common_corpus


[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [ ]:
TF_2.shape

(8166, 5716)

In [ ]:
TF_3 = TF_2.loc[TF_2.apply(sum,axis=1) > 0 ,:]

TF Table을 요약하여 각 상품명에서 몇 번째 단어가 몇 번 쓰였는지를 튜플 형태로 저장한 lst_all이라는 리스트를 만들었다.

In [ ]:
def make_doc(row):
    lst = []
    for idx,i in enumerate(row):
        if i == 0:
            pass
        else:
            lst.append((idx,i))
    return lst

lst_all = TF_3.apply(make_doc,axis=1)
    
            

In [ ]:
lst_all = [i for i in lst_all]

In [ ]:
lst_all

[[(5052, 1)],
 [(396, 1), (966, 1), (2363, 3)],
 [(966, 1), (3003, 1), (3004, 1)],
 [(4672, 1), (5052, 1)],
 [(292, 1), (2675, 2)],
 [(431, 1), (4742, 2)],
 [(1440, 1), (4638, 2)],
 [(431, 1), (4742, 2)],
 [(428, 1), (3003, 1), (3004, 1), (3582, 1), (3779, 1), (5153, 1)],
 [(1439, 1), (5052, 2)],
 [(4958, 2)],
 [(2675, 1)],
 [(966, 1), (1334, 2)],
 [(2310, 1), (4442, 1), (4958, 2)],
 [(966, 1), (1334, 2)],
 [(1439, 1), (5052, 2)],
 [(1439, 1), (1757, 2)],
 [(429, 1), (3584, 1), (3782, 2)],
 [(429, 1), (3045, 1), (3157, 1), (3180, 1), (3886, 1)],
 [(3431, 1)],
 [(1526, 1), (2624, 1), (3510, 1), (4653, 1), (4654, 1)],
 [(1526, 1), (3510, 1), (4052, 1), (4533, 1), (4653, 1), (4654, 1)],
 [(294, 1), (429, 1), (430, 1), (4533, 1)],
 [(1901, 1)],
 [(3810, 1)],
 [(2137, 1), (4457, 1)],
 [(1069, 1), (5471, 1)],
 [(1950, 1), (3302, 1), (3549, 1), (4620, 1), (4747, 1)],
 [(1950, 1), (3302, 1), (3549, 1), (4620, 1)],
 [(2675, 2), (2979, 1), (3029, 1), (4564, 1)],
 [(428, 1), (2328, 1), (3003, 1),

각 Topic별 비율을 기반으로 분류될 확률을 doc_pro 변수에 저장했다.

In [ ]:
a = pd.DataFrame(TF_3.index.value_counts() / TF_3.index.value_counts().sum()).reset_index()['Main_category'].tolist()

doc_pro = [(i,j) for i,j in enumerate(a)]

In [ ]:
doc_pro

[(0, 0.1512107161257084),
 (1, 0.12712519319938176),
 (2, 0.12686759402369913),
 (3, 0.11038124678001031),
 (4, 0.0926069036579083),
 (5, 0.09234930448222566),
 (6, 0.07959814528593509),
 (7, 0.07869654817104585),
 (8, 0.06298299845440494),
 (9, 0.03799587841318908),
 (10, 0.025373518804739825),
 (11, 0.014811952601751674)]

In [ ]:
lda = LdaModel(lst_all ,num_topics=12)

### LDA 시각화

In [ ]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()



df1 = pd.read_csv('product.csv',header = None)

df_title =  df1.iloc[:,[3]]
df_title.columns = ['title']

df_title = df_title.drop_duplicates()

def make_okt(row):
    row['title'] = okt.nouns(row['title'])
    return row
    
    
df_title_spec = df_title.apply(make_okt,axis=1)

In [ ]:
title_lst = df_title['title'].tolist()

In [ ]:
title_lst

[['파리', '바게뜨', '실키롤', '케익'],
 ['설빙', '설빙', '설빙', '디지털', '금액'],
 ['아웃백', '스테이크하우스', '아웃백', '디지털'],
 ['파리', '바게뜨', '블레싱', '초코', '베리', '케이크'],
 ['스타벅스', '스타벅스', '전용', '카드', '교환'],
 ['파리', '바게뜨', '생', '딸기', '요거트', '케이크'],
 ['스타벅스', '스타벅스', '전용', '카드', '교환'],
 ['쿠우', '쿠우', '쿠우', '쿠우', '프티', '카드'],
 ['커피빈', '커피빈', '모바일', '교환'],
 ['쿠우', '쿠우', '쿠우', '쿠우', '프티', '카드'],
 ['아웃백', '스테이크하우스', '아웃백', '이디야', '페이', '기프트', '외식'],
 ['파리', '바게뜨', '파리', '바게뜨', '모바일'],
 ['아웃백', '스테이크하우스', '아웃백', '디지털'],
 ['써브웨이', '써브웨이'],
 ['투썸플레이스', '투썸플레이스'],
 ['스타벅스', '아이스', '카페', '아메리카노', '잔', '생크림', '카스테라'],
 ['커피빈', '커피빈', '모바일', '교환'],
 ['맥도날드', '맥도날드', '디지털'],
 ['메', '커피', '메', '커피'],
 ['투썸플레이스', '투썸플레이스', '스트로베리', '초콜릿', '생크림'],
 ['맥도날드', '맥도날드', '디지털'],
 ['파리', '바게뜨', '파리', '바게뜨', '모바일'],
 ['배스킨라빈스', '배스킨라빈스', '모바일'],
 ['스타벅스', '스타벅스', '힘', '나', '세트'],
 ['이랜드', '이랜드', '외식', '통합', '기프트카드'],
 ['일본', '아이튠즈', '기프트카드', '애플', '앱스토어'],
 ['롯데', '시네마', '관람', '예매', '옵션'],
 ['컬쳐', '랜드', '문화', '상품권', '스마일', '클럽', '컬쳐', '랜드',

In [ ]:
processed_data

[['일본',
  '동경',
  '박열',
  '흑도회',
  '기관지',
  '흑도',
  '흑우회',
  '기관지',
  '태선인',
  '현사회',
  '간행',
  '무정부주의',
  '선전',
  '회원',
  '규합',
  '노력',
  '박열',
  '주도',
  '무정부주의',
  '단체',
  '불령사',
  '가입',
  '활동',
  '동년',
  '동경',
  '대지진',
  '직후',
  '체포',
  '박열',
  '대역사건',
  '기소',
  '사형',
  '이후',
  '무기징역',
  '감형',
  '우쓰노미야형무소',
  '우도궁형무소',
  '도치기지소',
  '회목지소',
  '수감',
  '옥중',
  '순국',
  '박열',
  '의기투합',
  '일본',
  '제국주의',
  '반대',
  '활동',
  '시작',
  '흑도회',
  '가입',
  '박열',
  '기관지',
  '흑도',
  '발간',
  '책임',
  '흑우회',
  '조직',
  '사상연구',
  '연설회',
  '추진',
  '기관지',
  '민중운동',
  '한글',
  '발행',
  '후토이센진',
  '창간',
  '현사회',
  '이름',
  '개칭',
  '지속',
  '발간',
  '무정부주의',
  '운동가',
  '규합',
  '불령사',
  '조직',
  '불령사',
  '한국',
  '형평운동',
  '지지',
  '한편',
  '노동운동',
  '적극',
  '지원',
  '무정부주의',
  '입각',
  '항일투쟁',
  '전개',
  '의열단',
  '의열투쟁',
  '추진',
  '폭탄',
  '도쿄',
  '반입',
  '일왕',
  '정부',
  '요인',
  '암살',
  '계획',
  '도쿄대지진',
  '직후',
  '보호',
  '감속',
  '명목',
  '도쿄',
  '세타가야',
  '경찰서',
  '구금',
  '취조',
  '도중',
  '폭탄',
  '반입',
  '사실',
  '대역사건',
  '주

In [ ]:
title_lst = [i for i in df_title_spec['title'] if len(i) >= 1]

In [ ]:
len(dictionary)

1446

In [ ]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
import logging
import pickle
import pyLDAvis.gensim_models
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt


if __name__ == '__main__':


    # 정수 인코딩과 빈도수 생성
    dictionary = corpora.Dictionary(title_lst)

    # 출현빈도가 적거나 자주 등장하는 단어는 제거
    dictionary.filter_extremes(no_below=10, no_above=0.05)
    
    corpus = [dictionary.doc2bow(text) for text in title_lst]
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))
    
    perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
    coherence_logger = CoherenceMetric(corpus=corpus, coherence="u_mass", logger='shell')



    lda_model = LdaModel(corpus, id2word = dictionary,alpha=a, num_topics=12, passes=30, callbacks=[coherence_logger, perplexity_logger])
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=title_lst, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score (c_v): ', coherence_lda)

    # Compute Coherence Score using UMass
    coherence_model_lda = CoherenceModel(model=lda_model, texts=title_lst, dictionary=dictionary, coherence="u_mass")
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score (u_mass): ', coherence_lda)


    # 저장
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('lda_dictionary.gensim')
    lda_model.save('lda_model.gensim')

    # pyLDAvis html 저장
    lda_visualization = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
    pyLDAvis.save_html(lda_visualization, 'gmarket_lda_2.html')
    pyLDAvis.display(lda_visualization)



2022-06-06 20:48:35,446 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-06-06 20:48:35,567 : INFO : built Dictionary(9779 unique tokens: ['바게뜨', '실키롤', '케익', '파리', '금액']...) from 9929 documents (total 76651 corpus positions)
2022-06-06 20:48:35,568 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(9779 unique tokens: ['바게뜨', '실키롤', '케익', '파리', '금액']...) from 9929 documents (total 76651 corpus positions)", 'datetime': '2022-06-06T20:48:35.568048', 'gensim': '4.1.2', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2022-06-06 20:48:35,578 : INFO : discarding 8333 tokens: [('실키롤', 1), ('케익', 2), ('설빙', 8), ('스테이크하우스', 8), ('아웃백', 8), ('블레싱', 1), ('초코', 4), ('케이크', 5), ('요거트', 5), ('쿠우', 3)]...
2022-06-06 20:48:35,579 : INFO : keeping 1446 tokens which were in no less than 10 and no more than 496 (=5.0%) documents
2022-06-06 20:48:35,583 : INFO : resulting dictionary:

Number of unique tokens: 1446
Number of documents: 9929


2022-06-06 20:48:36,080 : INFO : merging changes from 2000 documents into a model of 9929 documents
2022-06-06 20:48:36,082 : INFO : topic #11 (0.015): 0.039*"노트" + 0.032*"갤럭시" + 0.026*"종" + 0.026*"샌들" + 0.026*"여름" + 0.026*"여성" + 0.026*"북" + 0.019*"가방" + 0.019*"남성" + 0.019*"힙색"
2022-06-06 20:48:36,083 : INFO : topic #10 (0.025): 0.055*"남자" + 0.051*"양말" + 0.030*"발목" + 0.030*"지갑" + 0.026*"용" + 0.026*"켤레" + 0.021*"여행" + 0.021*"여자" + 0.021*"카드" + 0.017*"구두"
2022-06-06 20:48:36,083 : INFO : topic #2 (0.127): 0.033*"가방" + 0.021*"여행" + 0.017*"남성" + 0.017*"남자" + 0.015*"운동화" + 0.014*"여성" + 0.013*"크로스" + 0.013*"카드" + 0.011*"용" + 0.011*"스니커즈"
2022-06-06 20:48:36,084 : INFO : topic #1 (0.127): 0.023*"용" + 0.021*"캐리어" + 0.020*"여행" + 0.017*"여성" + 0.015*"남성" + 0.014*"벨트" + 0.013*"집" + 0.013*"종" + 0.012*"가방" + 0.010*"블루투스"
2022-06-06 20:48:36,084 : INFO : topic #0 (0.151): 0.030*"상품권" + 0.016*"모바일" + 0.014*"문화" + 0.013*"용" + 0.012*"카드" + 0.012*"닌텐도" + 0.010*"금액" + 0.010*"가방" + 0.009*"가능" + 0.009*"컬쳐"



Coherence Score (c_v):  0.5243655733133321

Coherence Score (u_mass):  -15.311822470621697


In [ ]:
a

[0.1512107161257084,
 0.12712519319938176,
 0.12686759402369913,
 0.11038124678001031,
 0.0926069036579083,
 0.09234930448222566,
 0.07959814528593509,
 0.07869654817104585,
 0.06298299845440494,
 0.03799587841318908,
 0.025373518804739825,
 0.014811952601751674]

LDA에 의해 임의로 선정된 topic 사이의 거리를 표현한 Intertopic Distance Map

In [ ]:
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('lda_dictionary.gensim')
    lda_model.save('lda_model.gensim')

    # pyLDAvis html 저장
    lda_visualization = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
    pyLDAvis.save_html(lda_visualization, 'gmarket_lda_2.html')
    pyLDAvis.show(lda_visualization)


2022-06-06 20:38:26,424 : INFO : Dictionary lifecycle event {'fname_or_handle': 'lda_dictionary.gensim', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-06-06T20:38:26.424110', 'gensim': '4.1.2', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2022-06-06 20:38:26,427 : INFO : saved lda_dictionary.gensim
2022-06-06 20:38:26,428 : INFO : LdaState lifecycle event {'fname_or_handle': 'lda_model.gensim.state', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-06-06T20:38:26.428111', 'gensim': '4.1.2', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2022-06-06 20:38:26,429 : INFO : saved lda_model.gensim.state
2022-06-06 20:38:26,431 : INFO : LdaModel lifecycle event {'fname_or_handle': 'lda_model.gensim', 'separately': "['expElogbeta', '

### L-LDA 시각화

In [ ]:
lda_visualization = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, 
                                                   sort_topics=False,
                                                    doc_topic_dist= csr_matrix(df_2.values),
                                                  doc_lengths = df_2.apply(sum,axis=0),)
pyLDAvis.save_html(lda_visualization, 'gmarket_lda_final.html',)
pyLDAvis.display(lda_visualization)

### PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=750, random_state=7)
X_train_pca = pca.fit_transform(X_train_tfidf_df_2.values)
X_test_pca = pca.transform(X_test_tfidf_df_2.values)

print('Original tfidf matrix shape:', X_train_tfidf_df_2.shape)
print('PCA Converted matrix shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))

Original tfidf matrix shape: (7974, 1465)
PCA Converted matrix shape: (7974, 750)
Sum of explained variance ratio: 0.805


In [ ]:
X_test_pca

array([[-1.52019772e-02,  4.89462045e-04, -3.94749945e-03, ...,
         7.97662200e-03, -2.35291713e-02,  2.77270594e-02],
       [-8.57996208e-03, -4.40624932e-02, -3.89154205e-02, ...,
         5.80769672e-03,  1.82299703e-03, -2.90254422e-03],
       [ 1.56973393e-03, -1.29087888e-02,  3.78064710e-04, ...,
        -8.70912753e-03, -2.09783304e-04,  2.68713726e-03],
       ...,
       [-1.15638180e-02, -7.01436144e-03,  7.07333707e-04, ...,
        -2.43882012e-04,  2.94628995e-06,  7.97855762e-04],
       [-1.26072067e-02, -8.13864354e-03,  1.90496402e-04, ...,
         3.26370026e-02,  4.68216942e-03, -1.15286009e-03],
       [-1.71025573e-02, -2.42766830e-02, -1.34941066e-02, ...,
         2.90216248e-02,  3.89002516e-02, -2.83207097e-02]])

#### GradientBoostingCalssifier에 PCA로 얻은 주성분을 input으로 넣은 결과

In [ ]:
from sklearn.decomposition import PCA
gd = GradientBoostingClassifier()

gd.fit(X_train_pca, y_train)

GradientBoostingClassifier()

In [ ]:
import joblib 
joblib.dump(gd, 'Gradient_b.pkl')

In [ ]:
print('#Train set score: {:.3f}'.format(gd.score(X_train_pca, y_train)))
print('#Test set score: {:.3f}'.format(gd.score(X_test_pca, y_test))) 

#Train set score: 0.809
#Test set score: 0.656


#### GridSearchCV를 통해 하이퍼파라미터를 튜닝 후 RandomForestClassifier에 PCA로 얻은 주성분을 input으로 넣은 결과

In [ ]:
from sklearn.preprocessing import MinMaxScaler #fixed import

#ValueError: Negative values in data passed to MultinomialNB (input X) 에러 방지
scaler = MinMaxScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)


n_estimators = [10, 50]
max_depth = [3, None] 
max_features = [0.1, 0.2, 0.5] # feature 수
min_samples_split = [2, 4, 6] # 노드를 분할하기 위한 최소 샘플 수
min_samples_leaf = [2, 4, 6] # 리프 노드가 되기 위해 필요한 최소 샘플 수

hyperparams = {
    'n_estimators': n_estimators, 
    'max_depth': max_depth, 
    'max_features': max_features,
    'min_samples_split': min_samples_split, 
    'min_samples_leaf': min_samples_leaf
}

RF=GridSearchCV(
    estimator = RandomForestClassifier(random_state=1), 
    param_grid = hyperparams, 
    verbose=True, 
    cv=5, 
    scoring = "accuracy", 
    n_jobs=-1
)


RF.fit(X_train_pca, y_train)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1), n_jobs=-1,
             param_grid={'max_depth': [3, None],
                         'max_features': [0.1, 0.2, 0.5],
                         'min_samples_leaf': [2, 4, 6],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [10, 50]},
             scoring='accuracy', verbose=True)

In [ ]:
print('#Train set score: {:.3f}'.format(RF.score(X_train_pca, y_train)))
print('#Test set score: {:.3f}'.format(RF.score(X_test_pca, y_test))) 


#Train set score: 0.866
#Test set score: 0.675
